In [1]:
import json
from datasets import load_dataset
import plotly.express as px
#visualize
import matplotlib.pyplot as plt
import os
import pandas as pd

%matplotlib inline

os.chdir('/Users/jon/Documents/DSDM/term_2/adv_meth_nlp/nlp-final/')

/Users/jon/miniconda3/envs/nlp-final/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data exploration

First load the dataset.

In [2]:
import json
from datasets import load_dataset

# Load the English part of the dataset
train_dataset = load_dataset('nlpaueb/multi_eurlex', 'en', split='train')
test_dataset = load_dataset('nlpaueb/multi_eurlex', 'en', split='test')
dev_dataset = load_dataset('nlpaueb/multi_eurlex', 'en', split='dev')

Found cached dataset multi_eurlex (/Users/jon/.cache/huggingface/datasets/nlpaueb___multi_eurlex/en/1.0.0/1addee7110a20c2b01cc3de89456786482e4eea1d2ead0bea3d5383b16cc9fce)


In [4]:
# Load (label_id, descriptor) mapping 
with open('./data/eurovoc_descriptors.json') as jsonl_file:
    eurovoc_descriptors =  json.load(jsonl_file)

# Get feature map info
train_classlabel = train_dataset.features["labels"].feature
test_classlabel = test_dataset.features["labels"].feature

# Load (label_id, descriptor) mapping 
with open('./data/eurovoc_concepts.json') as jsonl_file:
    eurovoc_concepts =  json.load(jsonl_file)


In [10]:

# Retrieve IDs and descriptors from dataset
train_agg_df = []
def get_agg_df(dataset:pd.DataFrame):

    for i, sample in enumerate(dataset):    
        for label_id in sample['labels']:
            result = {}
            result['celex_id'] = sample['celex_id']
            result['label_id'] = label_id
            result['eurovoc_id'] = classlabel.int2str(label_id)
            result['eurovoc_desc'] = eurovoc_descriptors[classlabel.int2str(label_id)]['en']
            df_agg.append(result)
            # Get the EuroVoc level for the current label ID
            for level, ids in eurovoc_concepts.items():
                if classlabel.int2str(label_id) in ids:
                    result['eurovoc_level'] = level
                    break
    df_agg = pd.DataFrame(df_agg)
    return(df_agg)
df_agg.head()



,celex_id,label_id,eurovoc_id,eurovoc_desc,eurovoc_level
0,32003R1012,2,100148,finance,level_1
1,32003R1012,5,100143,international relations,level_1
2,32003R1012,10,100142,politics,level_1
3,32003R1012,8,100154,transport,level_1
4,32003R1012,3,100147,trade,level_1


In [19]:
df_agg['eurovoc_level'].value_counts()

level_1    35286
Name: eurovoc_level, dtype: int64

In [24]:
# Group by eurovoc_level and eurovoc_desc and count occurrences
df_grouped = df_agg\
    .groupby(['eurovoc_desc'])\
    .size()\
    .reset_index(name='count')\
    .sort_values(by='count', ascending=False)


df_grouped

,eurovoc_desc,count
19,trade,6121
1,agri-foodstuffs,5439
10,geography,4821
2,"agriculture, forestry and fisheries",3805
0,EUROPEAN UNION,2556
9,finance,1728
13,international relations,1709
4,economics,1492
11,industry,1279
18,social questions,1077


In [20]:
train = load_dataset('nlpaueb/multi_eurlex', 'en', split="train").to_pandas()
validation = load_dataset('nlpaueb/multi_eurlex', 'en', split="validation").to_pandas()
test = load_dataset('nlpaueb/multi_eurlex', 'en', split="test").to_pandas()

Found cached dataset multi_eurlex (/Users/jon/.cache/huggingface/datasets/nlpaueb___multi_eurlex/en/1.0.0/1addee7110a20c2b01cc3de89456786482e4eea1d2ead0bea3d5383b16cc9fce)
Found cached dataset multi_eurlex (/Users/jon/.cache/huggingface/datasets/nlpaueb___multi_eurlex/en/1.0.0/1addee7110a20c2b01cc3de89456786482e4eea1d2ead0bea3d5383b16cc9fce)
Found cached dataset multi_eurlex (/Users/jon/.cache/huggingface/datasets/nlpaueb___multi_eurlex/en/1.0.0/1addee7110a20c2b01cc3de89456786482e4eea1d2ead0bea3d5383b16cc9fce)


In [21]:
train.head()

,celex_id,text,labels
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,"[2, 5, 10, 8, 3, 18, 15]"
1,32008R0284,COMMISSION REGULATION (EC) No 284/2008\nof 27 ...,"[3, 18, 17, 6]"
2,32004D0761,COMMISSION DECISION\nof 26 October 2004\nlayin...,"[17, 19, 10, 6, 15]"
3,31992R2310,COMMISSION REGULATION (EEC) No 2310/92 of 31 J...,"[7, 3, 17, 6]"
4,31995R0423,COUNCIL REGULATION (EC) No 423/95 of 20 Februa...,"[17, 19, 6]"


## Data and labels

### Eurovoc concepts

`eurovoc_id` is the ID of a certain area of government activity.

`level` refers to the specificity of a certain topic. For example, european parliament is level 0, quantum computing is level 7.

In [22]:
# Load (label_id, descriptor) mapping 
with open('./data/eurovoc_concepts.json') as jsonl_file:
    eurovoc_concepts =  json.load(jsonl_file)

df_concepts = []

for level, ids in eurovoc_concepts.items():
    for id in ids:
        results = {'eurovoc_id': id, 'level' : level}
        df_concepts.append(results)
df_concepts = pd.DataFrame(df_concepts)

df_concepts.head()

,eurovoc_id,level
0,100149,level_1
1,100160,level_1
2,100148,level_1
3,100147,level_1
4,100152,level_1


### Eurovoc descriptors

In [23]:

with open('./data/eurovoc_descriptors.json') as jsonl_file:
    eurovoc_descriptors =  json.load(jsonl_file)

df_descriptors = []
for id, label in eurovoc_descriptors.items():
    results = {'eurovoc_id':id, 'descriptor':label['en']}
    df_descriptors.append(results)
df_descriptors = pd.DataFrame(df_descriptors)
df_descriptors

,eurovoc_id,descriptor
0,100149,social questions
1,100160,industry
2,100148,finance
3,100147,trade
4,100152,business and competition
...,...,...
7385,199,second Lomé Convention
7386,4053,third Lomé Convention
7387,198,first Lomé Convention
7388,4355,Sysmin


As we can observe lables are not equally distributed over documents. Top three categories are EUROVOC concepts 3 (adhesive), 18 (parliamentary committee), 6 (wholesale trade)

In [27]:
# Load (label_id, descriptor) mapping 
with open('./data/eurovoc_descriptors.json') as jsonl_file:
    eurovoc_concepts =  json.load(jsonl_file)

# Get feature map info
classlabel = train.features["labels"].feature

# Retrieve IDs and descriptors from dataset
for sample in train:
  print(f'DOCUMENT: {sample["celex_id"]}')
  # DOCUMENT: 32006D0213
  for label_id in sample['labels']:
    print(f'LABEL: id:{label_id}, eurovoc_id: {classlabel.int2str(label_id)}, \
            eurovoc_desc:{eurovoc_concepts[classlabel.int2str(label_id)]}')
    # LABEL: id: 1, eurovoc_id: '100160', eurovoc_desc: 'industry'

AttributeError: 'DataFrame' object has no attribute 'features'

## Baseline (ie non ML solution) + metrics on test set


As baseline we can take each sentence and get most common words used in the text and predict based on that.

In [10]:
text_only = train_exp.groupby('labels', group_keys=False)['text'].apply(lambda x: x.str.split())

In [11]:
train_exp.head()

,celex_id,text,labels
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,2
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,5
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,10
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,8
0,32003R1012,Commission Regulation (EC) No 1012/2003\nof 12...,3


In [17]:
concepts_df = []
for i, v in eurovoc_concepts.items():
    result = {'concept_id': i, 'label': v['en']}
    concepts_df.append(result)
concepts_df = pd.DataFrame(concepts_df)
concepts_df.head()

,concept_id,label
0,100149,social questions
1,100160,industry
2,100148,finance
3,100147,trade
4,100152,business and competition


In [18]:
concepts_df['label']

NameError: name 'concepts_df' is not defined

In [17]:
from datasets import load_dataset
dataset = load_dataset('nlpaueb/multi_eurlex', 'all_languages')


AttributeError: 'MultiEURLEX' object has no attribute 'languages'

## RNN solution + metrics on test set


## BERT model + metrics on test set